In [7]:
# !pip install autokeras


In [1]:
import pandas as pd
import tensorflow as tf

import autokeras as ak


To make this tutorial easy to follow, we use the UCI Airquality dataset, and try to
forecast the AH value at the different timesteps. Some basic preprocessing has also
been performed on the dataset as it required cleanup.

## A Simple Example
The first step is to prepare your data. Here we use the [UCI Airquality
dataset](https://archive.ics.uci.edu/ml/datasets/Air+Quality) as an example.


In [12]:
dataset = tf.keras.utils.get_file(
    fname="AirQualityUCI.csv",
    origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00360/"
    "AirQualityUCI.zip",
    extract=True,
)


In [14]:
dataset = pd.read_csv("./AirQualityUCI.csv", sep=";")
dataset.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [15]:
dataset = dataset[dataset.columns[:-2]]
dataset = dataset.dropna()
dataset = dataset.replace(",", ".", regex=True)

val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

In [16]:
data_x = data_train[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

data_x_val = validation_data[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

# Data with train data and the unseen data from subsequent time steps.
data_x_test = dataset[
    [
        "CO(GT)",
        "PT08.S1(CO)",
        "NMHC(GT)",
        "C6H6(GT)",
        "PT08.S2(NMHC)",
        "NOx(GT)",
        "PT08.S3(NOx)",
        "NO2(GT)",
        "PT08.S4(NO2)",
        "PT08.S5(O3)",
        "T",
        "RH",
    ]
].astype("float64")

data_y = data_train["AH"].astype("float64")

data_y_val = validation_data["AH"].astype("float64")

print(data_x.shape)  # (6549, 12)
print(data_y.shape)  # (6549,)


(6549, 12)
(6549,)


The second step is to run the [TimeSeriesForecaster](/time_series_forecaster).
As a quick demo, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [17]:
predict_from = 1
predict_until = 10
lookback = 3
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    max_trials=1,
    objective="val_loss",
)
# Train the TimeSeriesForecaster with train data
clf.fit(
    x=data_x,
    y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32,
    epochs=10,
)
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))


Trial 1 Complete [00h 01m 00s]
val_loss: 2631.228759765625

Best val_loss So Far: 2631.228759765625
Total elapsed time: 00h 01m 00s
Epoch 1/10
205/205 [==============================] - 13s 28ms/step - loss: 898.7586 - mean_squared_error: 898.7586 - val_loss: 3091.3425 - val_mean_squared_error: 3091.3425
Epoch 2/10
205/205 [==============================] - 3s 15ms/step - loss: 886.8903 - mean_squared_error: 886.8903 - val_loss: 3060.8379 - val_mean_squared_error: 3060.8379
Epoch 3/10
205/205 [==============================] - 3s 15ms/step - loss: 866.6203 - mean_squared_error: 866.6203 - val_loss: 3014.9285 - val_mean_squared_error: 3014.9285
Epoch 4/10
205/205 [==============================] - 3s 15ms/step - loss: 844.1583 - mean_squared_error: 844.1583 - val_loss: 2971.8030 - val_mean_squared_error: 2971.8030
Epoch 5/10
205/205 [==============================] - 3s 15ms/step - loss: 825.6188 - mean_squared_error: 825.6188 - val_loss: 2926.8142 - val_mean_squared_error: 2926.8142
Ep

293/293 [==============================] - 3s 10ms/step
(10, 1)
88/88 [==============================] - 3s 8ms/step - loss: 2772.7075 - mean_squared_error: 2772.7075
[2772.70751953125, 2772.70751953125]
